In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [5]:
train_path = r"C:\Users\jayvinit\Desktop\Group_Project_Smart_Internz\Model_data\train"
test_path = r"C:\Users\jayvinit\Desktop\Group_Project_Smart_Internz\Model_data\test"

In [6]:
train_datagen = ImageDataGenerator(rescale = 1/255,zoom_range=0.2, shear_range = 0.2)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [7]:
train = train_datagen.flow_from_directory(train_path, target_size=(232,232),batch_size=32)
test = test_datagen.flow_from_directory(test_path, target_size=(232,232),batch_size=32)

Found 4186 images belonging to 3 classes.
Found 1044 images belonging to 3 classes.


In [8]:
import numpy as np

# Get true labels
true_labels = []

num_steps = len(test)
for i in range(num_steps):
    images, labels = test[i]
    true_labels.extend(np.argmax(labels, axis=1))

true_labels = np.array(true_labels)

VGG 16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten


In [ ]:
conv_base = VGG16(weights='imagenet',include_top =False,input_shape=(232,232,3))

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 232, 232, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 232, 232, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 232, 232, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 116, 116, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 116, 116, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 116, 116, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 58, 58, 128)       0     

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(3,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 21138243 (80.64 MB)
Trainable params: 21138243 (80.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Freezing the convolutional base
conv_base.trainable=False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 21138243 (80.64 MB)
Trainable params: 6423555 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
#compiling the mode.
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [ ]:
class StopTrainingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') >= 0.990:
            print("\nValidation accuracy reached 80.0% or higher. Stopping training.")
            self.model.stop_training = True

In [ ]:
#fit the model
model.fit(train,epochs=10,
          validation_data = test,
          validation_steps= len(test)//32,
          callbacks=[StopTrainingCallback()])
model.save('/content/drive/MyDrive/Covid data/Models/train_2.h5')


Epoch 1/10
131/131 [==============================] - 1800s 14s/step - loss: 0.2812 - accuracy: 0.8982 - val_loss: 0.3894 - val_accuracy: 0.8750
Epoch 2/10
131/131 [==============================] - 1798s 14s/step - loss: 0.1477 - accuracy: 0.9522 - val_loss: 0.4043 - val_accuracy: 0.9062
Epoch 3/10
131/131 [==============================] - ETA: 0s - loss: 0.1179 - accuracy: 0.9651 
Validation accuracy reached 80.0% or higher. Stopping training.
131/131 [==============================] - 1799s 14s/step - loss: 0.1179 - accuracy: 0.9651 - val_loss: 0.0737 - val_accuracy: 1.0000


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [9]:
from tensorflow.keras.models import load_model
model = load_model('train_2.h5')

In [10]:
predictions = model.predict(test)
predicted_class = np.argmax(predictions, axis=1)

: 

In [ ]:
print(train.class_indices)

{'COVID': 0, 'NORMAL': 1, 'PNEUMONIA': 2}


In [ ]:

img = image.load_img('/content/drive/MyDrive/Covid data/Model_data/test/PNEUMONIA/PNEUMONIA_1449.png',target_size=(232,232))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)
pred_class = np.argmax(pred, axis=1)
index = ['COVID','NORMAL','PNEUMONIA']
str(index[pred_class[0]])

1/1 [==============================] - 0s 396ms/step


'PNEUMONIA'

In [ ]:
x.ndim

In [ ]:
_, train_accuracy = model.evaluate(train)

131/131 [==============================] - 1779s 14s/step - loss: 0.0756 - accuracy: 0.9768


In [ ]:
print(f'Test accuracy: {train_accuracy* 100:.2f}%')

Test accuracy: 97.68%


In [ ]:
test_loss, test_accuracy = model.evaluate(test)

33/33 [==============================] - 444s 13s/step - loss: 0.1425 - accuracy: 0.9521


In [ ]:
print(f'Test accuracy: {test_accuracy* 100:.2f}%')

Test accuracy: 95.21%
